# Cross-validation for NN and Esemble model

In [1]:
import os
import pandas as pd
import numpy as np
import scipy.io as scio
from matplotlib import pyplot as plt
import seaborn as sns
import time
from datetime import datetime
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.externals import joblib
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import xgboost
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy.io import loadmat
import scipy

E:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
import tensorflow
import keras
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Input, Dropout
from keras.layers import BatchNormalization
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
class load():
    def load_data(filename):
        raw_data = pd.read_csv(filename)
        raw_data['filename'] = [str(i).zfill(4)+'.jpg' for i in raw_data['Index'].tolist()]
        raw_data['pointsname'] = [str(i).zfill(4)+'.mat' for i in raw_data['Index'].tolist()]
        return raw_data

    #read points data from mat data
    def load_points(points_path,data):
        n = data.shape[0]
        points_data = np.zeros([n,3003,2])
        start_time = time.time()
        for i in range(n):
            result = loadmat(points_path+data['pointsname'][i])
            key = sorted(result.keys())[-1]
            points = result[key]
            distance_h = []
            distance_v = []
            for d in range(points.shape[0]-1):
                for j in range(d+1,points.shape[0]):
                    distance_h.append(abs(points[d,0]-points[j,0]))
                    distance_v.append(abs(points[d,1]-points[j,1]))

            points_data[i,:,0]=distance_h
            points_data[i,:,1]=distance_v
        print("--- %s seconds ---" % (time.time() - start_time))
        return points_data.reshape([2500,6006])

In [4]:
path = '/Users/zhaoziqin/Desktop/train_set/'  # Please modify your own path
data = load.load_data(path+'label.csv')
points_path = '/Users/zhaoziqin/Desktop/train_set/points/'
X = load.load_points(points_path,data)
X = np.round(X,0)
y= data['emotion_idx'].to_numpy()

--- 24.325836181640625 seconds ---


In [6]:
Y = to_categorical(y)
Y = Y[:,1:]

## NN CV

In [12]:
k=10
train_accuracy=[]
test_accuracy=[]
for i in range(k):
    X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2)
    
    input_shape = [6006] 
    input_layer = Input(input_shape)
    x = BatchNormalization(momentum = 0.88)(input_layer) 
    x = Dense(22*10,activation='relu',kernel_initializer=initializers.glorot_normal(seed=6))(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization()(x) 
    x = Dense(22*8,activation='relu',kernel_initializer=initializers.glorot_normal(seed=6))(x)
    x = Dropout(0.25)(x)
    x = Dense(22*4,activation='relu',kernel_initializer=initializers.glorot_normal(seed=6))(x)
    x = Dropout(0.25)(x)
    x = Dense(22*2,activation='relu',kernel_initializer=initializers.glorot_normal(seed=6))(x)
    output_layer = Dense(22,activation='softmax',kernel_initializer=initializers.glorot_normal(seed=6))(x) 
    model = Model(input_layer,output_layer)
    
    model.compile(loss='categorical_crossentropy',optimizer = Adam(lr=0.001),metrics=['accuracy'])
    model.fit(X_train,y_train,epochs=200)

    train_pred=np.argmax(model.predict(X_train), axis=1)
    train_accuracy.append(accuracy_score( np.argmax(y_train, axis=1), train_pred) )
   
    test_pred=np.argmax(model.predict(X_test), axis=1)
    test_accuracy.append(accuracy_score( np.argmax(y_test, axis=1), test_pred) )
    
print("Training Accuracy: %.2f%% (sd %.2f%%)" % (np.mean(train_accuracy)*100, np.std(train_accuracy)*100))
print("Testing Accuracy: %.2f%% (sd %.2f%%)" % (np.mean(test_accuracy)*100, np.std(test_accuracy)*100))



Epoch 1/200
2000/2000 [==============================] - 2s 852us/step - loss: 3.0331 - accuracy: 0.0830
Epoch 2/200
2000/2000 [==============================] - 1s 432us/step - loss: 2.6010 - accuracy: 0.1855
Epoch 3/200
2000/2000 [==============================] - 1s 441us/step - loss: 2.3322 - accuracy: 0.2400
Epoch 4/200
2000/2000 [==============================] - 1s 430us/step - loss: 2.1447 - accuracy: 0.2930
Epoch 5/200
2000/2000 [==============================] - 1s 461us/step - loss: 2.0016 - accuracy: 0.3475
Epoch 6/200
2000/2000 [==============================] - 1s 435us/step - loss: 1.8734 - accuracy: 0.3710
Epoch 7/200
2000/2000 [==============================] - 1s 433us/step - loss: 1.7954 - accuracy: 0.3840
Epoch 8/200
2000/2000 [==============================] - 1s 402us/step - loss: 1.7177 - accuracy: 0.4165
Epoch 9/200
2000/2000 [==============================] - 1s 417us/step - loss: 1.6135 - accuracy: 0.4350
Epoch 10/200
2000/2000 [==============================]

2000/2000 [==============================] - 1s 443us/step - loss: 0.2527 - accuracy: 0.9185
Epoch 154/200
2000/2000 [==============================] - 1s 447us/step - loss: 0.2026 - accuracy: 0.9370
Epoch 155/200
2000/2000 [==============================] - 1s 456us/step - loss: 0.2782 - accuracy: 0.9135
Epoch 156/200
2000/2000 [==============================] - 1s 435us/step - loss: 0.2075 - accuracy: 0.9340
Epoch 157/200
2000/2000 [==============================] - 1s 431us/step - loss: 0.2145 - accuracy: 0.9325
Epoch 158/200
2000/2000 [==============================] - 1s 432us/step - loss: 0.2154 - accuracy: 0.9370
Epoch 159/200
2000/2000 [==============================] - 1s 426us/step - loss: 0.2283 - accuracy: 0.9250
Epoch 160/200
2000/2000 [==============================] - 1s 432us/step - loss: 0.2544 - accuracy: 0.9220
Epoch 161/200
2000/2000 [==============================] - 1s 431us/step - loss: 0.2377 - accuracy: 0.9230
Epoch 162/200
2000/2000 [==========================

2000/2000 [==============================] - 1s 384us/step - loss: 0.3914 - accuracy: 0.8685
Epoch 104/200
2000/2000 [==============================] - 1s 381us/step - loss: 0.3789 - accuracy: 0.87250s - loss: 0.3677 - accura
Epoch 105/200
2000/2000 [==============================] - 1s 377us/step - loss: 0.3729 - accuracy: 0.8790
Epoch 106/200
2000/2000 [==============================] - 1s 367us/step - loss: 0.3519 - accuracy: 0.8805
Epoch 107/200
2000/2000 [==============================] - 1s 367us/step - loss: 0.3179 - accuracy: 0.8920
Epoch 108/200
2000/2000 [==============================] - 1s 365us/step - loss: 0.3466 - accuracy: 0.8930
Epoch 109/200
2000/2000 [==============================] - 1s 364us/step - loss: 0.3174 - accuracy: 0.8995
Epoch 110/200
2000/2000 [==============================] - 1s 363us/step - loss: 0.3197 - accuracy: 0.8925
Epoch 111/200
2000/2000 [==============================] - 1s 377us/step - loss: 0.3101 - accuracy: 0.9055
Epoch 112/200
2000/2000 [

2000/2000 [==============================] - 1s 366us/step - loss: 0.6859 - accuracy: 0.76750s - loss: 0.6846 - accuracy: 0.76
Epoch 54/200
2000/2000 [==============================] - 1s 366us/step - loss: 0.6783 - accuracy: 0.7625
Epoch 55/200
2000/2000 [==============================] - 1s 367us/step - loss: 0.6508 - accuracy: 0.7715
Epoch 56/200
2000/2000 [==============================] - 1s 385us/step - loss: 0.6508 - accuracy: 0.7735
Epoch 57/200
2000/2000 [==============================] - 1s 370us/step - loss: 0.6308 - accuracy: 0.7855
Epoch 58/200
2000/2000 [==============================] - 1s 378us/step - loss: 0.6176 - accuracy: 0.7875
Epoch 59/200
2000/2000 [==============================] - 1s 395us/step - loss: 0.6438 - accuracy: 0.7800
Epoch 60/200
2000/2000 [==============================] - 1s 388us/step - loss: 0.5966 - accuracy: 0.7930
Epoch 61/200
2000/2000 [==============================] - 1s 382us/step - loss: 0.5884 - accuracy: 0.7955
Epoch 62/200
2000/2000 [=

2000/2000 [==============================] - 1s 358us/step - loss: 2.1795 - accuracy: 0.2880
Epoch 5/200
2000/2000 [==============================] - 1s 358us/step - loss: 2.0411 - accuracy: 0.32250s - loss: 2.0383 - accu
Epoch 6/200
2000/2000 [==============================] - 1s 381us/step - loss: 1.8955 - accuracy: 0.3665
Epoch 7/200
2000/2000 [==============================] - 1s 362us/step - loss: 1.8553 - accuracy: 0.3775
Epoch 8/200
2000/2000 [==============================] - 1s 363us/step - loss: 1.7795 - accuracy: 0.3960
Epoch 9/200
2000/2000 [==============================] - 1s 360us/step - loss: 1.6515 - accuracy: 0.4290
Epoch 10/200
2000/2000 [==============================] - 1s 361us/step - loss: 1.6353 - accuracy: 0.4365
Epoch 11/200
2000/2000 [==============================] - 1s 368us/step - loss: 1.5789 - accuracy: 0.4600
Epoch 12/200
2000/2000 [==============================] - 1s 364us/step - loss: 1.5480 - accuracy: 0.4720
Epoch 13/200
2000/2000 [================

...

2000/2000 [==============================] - 1s 371us/step - loss: 0.2334 - accuracy: 0.9295
Epoch 161/200
2000/2000 [==============================] - 1s 374us/step - loss: 0.1946 - accuracy: 0.9355
Epoch 162/200
2000/2000 [==============================] - 1s 386us/step - loss: 0.2161 - accuracy: 0.9385
Epoch 163/200
2000/2000 [==============================] - 1s 370us/step - loss: 0.1979 - accuracy: 0.9375
Epoch 164/200
2000/2000 [==============================] - 1s 368us/step - loss: 0.2021 - accuracy: 0.9345
Epoch 165/200
2000/2000 [==============================] - 1s 364us/step - loss: 0.1928 - accuracy: 0.9390
Epoch 166/200
2000/2000 [==============================] - 1s 365us/step - loss: 0.1767 - accuracy: 0.9405
Epoch 167/200
2000/2000 [==============================] - 1s 370us/step - loss: 0.2390 - accuracy: 0.9285
Epoch 168/200
2000/2000 [==============================] - 1s 377us/step - loss: 0.2112 - accuracy: 0.9310
Epoch 169/200
2000/2000 [==========================

2000/2000 [==============================] - 1s 544us/step - loss: 0.3285 - accuracy: 0.8900
Epoch 112/200
2000/2000 [==============================] - 1s 537us/step - loss: 0.3543 - accuracy: 0.8815
Epoch 113/200
2000/2000 [==============================] - 1s 587us/step - loss: 0.3196 - accuracy: 0.8935
Epoch 114/200
2000/2000 [==============================] - 1s 549us/step - loss: 0.3173 - accuracy: 0.8940
Epoch 115/200
2000/2000 [==============================] - 1s 560us/step - loss: 0.2751 - accuracy: 0.8960
Epoch 116/200
2000/2000 [==============================] - 1s 524us/step - loss: 0.3126 - accuracy: 0.8995
Epoch 117/200
2000/2000 [==============================] - 1s 513us/step - loss: 0.3369 - accuracy: 0.8845
Epoch 118/200
2000/2000 [==============================] - 1s 534us/step - loss: 0.3246 - accuracy: 0.8970
Epoch 119/200
2000/2000 [==============================] - 1s 532us/step - loss: 0.2860 - accuracy: 0.9045
Epoch 120/200
2000/2000 [==========================

2000/2000 [==============================] - 1s 507us/step - loss: 0.6080 - accuracy: 0.7975
Epoch 63/200
2000/2000 [==============================] - 1s 465us/step - loss: 0.6073 - accuracy: 0.7945
Epoch 64/200
2000/2000 [==============================] - 1s 477us/step - loss: 0.5917 - accuracy: 0.7900
Epoch 65/200
2000/2000 [==============================] - 1s 495us/step - loss: 0.5547 - accuracy: 0.8100
Epoch 66/200
2000/2000 [==============================] - 1s 538us/step - loss: 0.5864 - accuracy: 0.79550s -
Epoch 67/200
2000/2000 [==============================] - 1s 481us/step - loss: 0.5958 - accuracy: 0.79700s - loss: 0.5627 - accura
Epoch 68/200
2000/2000 [==============================] - 1s 470us/step - loss: 0.5752 - accuracy: 0.8045
Epoch 69/200
2000/2000 [==============================] - 1s 472us/step - loss: 0.5721 - accuracy: 0.8065
Epoch 70/200
2000/2000 [==============================] - 1s 489us/step - loss: 0.5739 - accuracy: 0.8085
Epoch 71/200
2000/2000 [=====

2000/2000 [==============================] - 1s 578us/step - loss: 1.4904 - accuracy: 0.4825
Epoch 13/200
2000/2000 [==============================] - 1s 561us/step - loss: 1.4565 - accuracy: 0.4905
Epoch 14/200
2000/2000 [==============================] - 1s 593us/step - loss: 1.4331 - accuracy: 0.4995
Epoch 15/200
2000/2000 [==============================] - 1s 567us/step - loss: 1.4057 - accuracy: 0.5165
Epoch 16/200
2000/2000 [==============================] - 1s 552us/step - loss: 1.3799 - accuracy: 0.5275
Epoch 17/200
2000/2000 [==============================] - 1s 540us/step - loss: 1.3435 - accuracy: 0.5240
Epoch 18/200
2000/2000 [==============================] - 1s 549us/step - loss: 1.3085 - accuracy: 0.5485
Epoch 19/200
2000/2000 [==============================] - 1s 557us/step - loss: 1.2756 - accuracy: 0.5525
Epoch 20/200
2000/2000 [==============================] - 1s 545us/step - loss: 1.2034 - accuracy: 0.5795
Epoch 21/200
2000/2000 [==============================] - 1

2000/2000 [==============================] - 1s 584us/step - loss: 0.2074 - accuracy: 0.9315
Epoch 165/200
2000/2000 [==============================] - 1s 586us/step - loss: 0.1854 - accuracy: 0.9390
Epoch 166/200
2000/2000 [==============================] - 1s 563us/step - loss: 0.1746 - accuracy: 0.9380
Epoch 167/200
2000/2000 [==============================] - 1s 573us/step - loss: 0.2087 - accuracy: 0.9365
Epoch 168/200
2000/2000 [==============================] - 1s 581us/step - loss: 0.1957 - accuracy: 0.9330
Epoch 169/200
2000/2000 [==============================] - 1s 584us/step - loss: 0.2045 - accuracy: 0.9285
Epoch 170/200
2000/2000 [==============================] - 1s 605us/step - loss: 0.2445 - accuracy: 0.9290
Epoch 171/200
2000/2000 [==============================] - 1s 629us/step - loss: 0.2273 - accuracy: 0.9360
Epoch 172/200
2000/2000 [==============================] - 1s 612us/step - loss: 0.2235 - accuracy: 0.93350s - loss: 0.2310 - ac
Epoch 173/200
2000/2000 [====

In [13]:
train_accuracy

[0.9985, 0.998, 0.9995, 0.999, 0.9995, 0.9995, 1.0, 0.9995, 0.9995, 0.999]

In [14]:
test_accuracy

[0.536, 0.54, 0.53, 0.55, 0.526, 0.552, 0.522, 0.53, 0.566, 0.534]

## Voting CV

In [15]:
lr = LogisticRegression(C=0.01, dual=False, fit_intercept=True,
                   intercept_scaling=1, max_iter=300,
                   multi_class='multinomial', penalty='l2',
                   solver='lbfgs', tol=0.0001)
svc = SVC(kernel= 'linear', random_state = 123, C = 0.00001)

lda = LDA(solver='eigen', shrinkage=.1, n_components=2)


In [21]:
k=10
train_accuracy=[]
test_accuracy=[]
for i in range(k):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    


    
    
    eclf = VotingClassifier(
    estimators=[('lr', lr), ('lda', lda), ('svm', svc)], 
    voting='hard',
    weights=[1.5,1.5,1])
    

    eclf_model = eclf.fit(X_train, y_train)

    
    train_pred=eclf.predict(X_train)
    train_accuracy.append(accuracy_score(y_train, train_pred) )
    
    test_pred=eclf.predict(X_test)
    test_accuracy.append(accuracy_score(y_test, test_pred) )
    
print("Training Accuracy: %.2f%% (sd %.2f%%)" % (np.mean(train_accuracy)*100, np.std(train_accuracy)*100))
print("Testing Accuracy: %.2f%% (sd %.2f%%)" % (np.mean(test_accuracy)*100, np.std(test_accuracy)*100))



E:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
E:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
E:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
E:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
E:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
E:\anaconda\lib\site-packages\sklearn\linear_model\logi

Training Accuracy: 83.57% (sd 0.73%)
Testing Accuracy: 53.48% (sd 1.85%)


In [22]:
train_accuracy

[0.8425, 0.8435, 0.8315, 0.8445, 0.834, 0.8355, 0.842, 0.824, 0.836, 0.8235]

In [23]:
test_accuracy

[0.538, 0.538, 0.51, 0.518, 0.522, 0.56, 0.542, 0.552, 0.508, 0.56]

# Saving

In [29]:
Voting_final = VotingClassifier(
    estimators=[('lr', lr), ('lda', lda), ('svm', svc)], 
    voting='hard',
    weights=[1.5,1.5,1])
Voting_final.fit(X,y)

E:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=0.01, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=300,
                                                 multi_class='multinomial',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('lda',
                              LinearDiscriminantAnalysis(n_components=2,
                                                         priors=None,
                                                         shrinkage...
                                

In [31]:
joblib.dump(Voting_final,'Voting_final.m')#please change the path

['Voting_final.m']

In [32]:
AAA=joblib.load('Voting_final.m')

In [33]:
accuracy_score(y, AAA.predict(X)) 

0.8152

In [34]:
input_shape = [6006] 
input_layer = Input(input_shape)
x = BatchNormalization(momentum = 0.88)(input_layer) 
x = Dense(22*10,activation='relu',kernel_initializer=initializers.glorot_normal(seed=6))(x)
x = Dropout(0.25)(x)
x = BatchNormalization()(x) 
x = Dense(22*8,activation='relu',kernel_initializer=initializers.glorot_normal(seed=6))(x)
x = Dropout(0.25)(x)
x = Dense(22*4,activation='relu',kernel_initializer=initializers.glorot_normal(seed=6))(x)
x = Dropout(0.25)(x)
x = Dense(22*2,activation='relu',kernel_initializer=initializers.glorot_normal(seed=6))(x)
output_layer = Dense(22,activation='softmax',kernel_initializer=initializers.glorot_normal(seed=6))(x) 
model = Model(input_layer,output_layer)
    
model.compile(loss='categorical_crossentropy',optimizer = Adam(lr=0.001),metrics=['accuracy'])


In [36]:
model.fit(X,Y,epochs=200)

Epoch 1/200
2500/2500 [==============================] - 2s 747us/step - loss: 2.9971 - accuracy: 0.1124
Epoch 2/200
2500/2500 [==============================] - 1s 389us/step - loss: 2.5818 - accuracy: 0.1800
Epoch 3/200
2500/2500 [==============================] - 1s 399us/step - loss: 2.2395 - accuracy: 0.2664
Epoch 4/200
2500/2500 [==============================] - 1s 388us/step - loss: 2.0388 - accuracy: 0.3104
Epoch 5/200
2500/2500 [==============================] - 1s 397us/step - loss: 1.9343 - accuracy: 0.3436
Epoch 6/200
2500/2500 [==============================] - 1s 387us/step - loss: 1.8728 - accuracy: 0.3656
Epoch 7/200
2500/2500 [==============================] - 1s 389us/step - loss: 1.7783 - accuracy: 0.3936
Epoch 8/200
2500/2500 [==============================] - 1s 409us/step - loss: 1.6985 - accuracy: 0.4100
Epoch 9/200
2500/2500 [==============================] - 1s 394us/step - loss: 1.6759 - accuracy: 0.4296
Epoch 10/200
2500/2500 [==============================]

2500/2500 [==============================] - 1s 462us/step - loss: 0.3980 - accuracy: 0.8700
Epoch 153/200
2500/2500 [==============================] - 1s 478us/step - loss: 0.3894 - accuracy: 0.8684
Epoch 154/200
2500/2500 [==============================] - 1s 481us/step - loss: 0.3625 - accuracy: 0.8804
Epoch 155/200
2500/2500 [==============================] - 1s 455us/step - loss: 0.3819 - accuracy: 0.87480s - loss: 0.4009 - 
Epoch 156/200
2500/2500 [==============================] - 1s 457us/step - loss: 0.3652 - accuracy: 0.8892
Epoch 157/200
2500/2500 [==============================] - 1s 451us/step - loss: 0.3794 - accuracy: 0.8792
Epoch 158/200
2500/2500 [==============================] - 1s 460us/step - loss: 0.3535 - accuracy: 0.8852
Epoch 159/200
2500/2500 [==============================] - 1s 453us/step - loss: 0.3379 - accuracy: 0.8912
Epoch 160/200
2500/2500 [==============================] - 1s 453us/step - loss: 0.3361 - accuracy: 0.8836
Epoch 161/200
2500/2500 [======

In [47]:
joblib.dump(model,'model_nn.m')#please change the path

['model_nn.m']

In [48]:
BBB=joblib.load('model_nn.m')

In [49]:
accuracy_score( np.argmax(Y, axis=1), np.argmax(BBB.predict(X), axis=1)) 

0.9948

In [56]:
data = {'Model':['NN','Voting'], 
        'Training Accuracy':['99.92% (sd 0.06%)', '83.57% (sd 0.73%)'],
        'Test Accuracy':['53.86% (sd 1.29%)', '53.48% (sd 1.85%)'],
       'fit CPU':['90%','71%  50% 14%'],
       'predict CPU':['"0"%','13.7%'],
       'fit memory':['+1GB','+2GB +1GB +1GB'],
       'predict memory':['"0"','+1GB'],
       'saved model size':['16mb','653mb']} 
pd.DataFrame(data)

,Model,Training Accuracy,Test Accuracy,fit CPU,predict CPU,fit memory,predict memory,saved model size
0,NN,99.92% (sd 0.06%),53.86% (sd 1.29%),90%,"""0""%",+1GB,"""0""",16mb
1,Voting,83.57% (sd 0.73%),53.48% (sd 1.85%),71% 50% 14%,13.7%,+2GB +1GB +1GB,+1GB,653mb
